In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Soowan Kim/Project/coinmarketcap_data_collection')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import csv
import time
from datetime import datetime, timedelta
import pickle
import re
import threading
import queue

***
# Authentification

In [ ]:
api_key = ''

with open('coinmarketcap_api_key.txt', 'r') as file:
    api_key = file.read().strip()
    print("API Key successfully loaded.")

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
}

API Key successfully loaded.


***
# Get Cryptocurrency Asset List

In [ ]:
# Read asset list if already collected
df_crypto_active_assets = pd.read_csv('data/crypto_active_assets.csv')
df_crypto_inactive_assets = pd.read_csv('data/crypto_inactive_assets.csv')
df_crypto_untracked_assets = pd.read_csv('data/crypto_untracked_assets.csv')
requested_assets = pd.read_excel('data/spaces_vote_link_v3.xlsx')
filtered_assets = requested_assets[requested_assets['UCID'].str.lower() != 'not sure'].dropna(subset=['UCID', 'DefiIlama_link'])
df_crypto_active_assets.head()

,id,rank,name,symbol,slug,is_active,first_historical_data,last_historical_data,platform
0,1,1,Bitcoin,BTC,bitcoin,1,2010-07-13T00:05:00.000Z,2024-10-13T07:45:00.000Z,NaN
1,2,22,Litecoin,LTC,litecoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,NaN
2,3,1150,Namecoin,NMC,namecoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,NaN
3,5,973,Peercoin,PPC,peercoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,NaN
4,6,8229,Novacoin,NVC,novacoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,NaN


In [ ]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'

# Get active assets
crypto_active_assets = requests.get(url, headers=headers, params={'listing_status': 'active'}).json()
# Get inactive assets (deprecated or delisted)
crypto_inactive_assets = requests.get(url, headers=headers, params={'listing_status': 'inactive'}).json()
# Get tracked assets
crypto_untracked_assets = requests.get(url, headers=headers, params={'listing_status': 'untracked'}).json()

# Read data and transform to dataframe
df_crypto_untracked_assets = pd.DataFrame(crypto_untracked_assets['data'])
df_crypto_active_assets = pd.DataFrame(crypto_active_assets['data'])
df_crypto_inactive_assets = pd.DataFrame(crypto_inactive_assets['data'])

# Combine the active and inactive DataFrames
df_crypto_tracked_assets = pd.concat([df_crypto_active_assets, df_crypto_inactive_assets], ignore_index=True)

# Save the DataFrame to a CSV file
df_crypto_active_assets.to_csv('data/df_crypto_active_assets.csv', index=False)
df_crypto_inactive_assets.to_csv('data/df_crypto_inactive_assets.csv', index=False)
df_crypto_untracked_assets.to_csv('data/crypto_untracked_assets.csv', index=False)

# Show the combined DataFrame
df_crypto_tracked_assets.head()

,id,rank,name,symbol,slug,is_active,first_historical_data,last_historical_data,platform
0,1,1,Bitcoin,BTC,bitcoin,1,2010-07-13T00:05:00.000Z,2024-10-13T07:45:00.000Z,None
1,2,22,Litecoin,LTC,litecoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,None
2,3,1150,Namecoin,NMC,namecoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,None
3,5,973,Peercoin,PPC,peercoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,None
4,6,8229,Novacoin,NVC,novacoin,1,2013-04-28T18:45:00.000Z,2024-10-13T07:45:00.000Z,None


#if you want to add period col...

In [ ]:
st = pd.to_datetime(df_crypto_active_assets['first_historical_data'], format='%Y-%m-%dT%H:%M:%S.%fZ')
ed = pd.to_datetime(df_crypto_active_assets['last_historical_data'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df_crypto_active_assets['period'] = ed - st
df_crypto_active_assets.describe()

,id,rank,is_active,period
count,9852.000000,9852.00000,9852.0,9852
mean,20314.457674,4926.50000,1.0,812 days 06:27:30.177730416
std,10392.680800,2844.17176,0.0,753 days 00:24:36.989101944
min,1.000000,1.00000,1.0,0 days 14:40:00
25%,10743.500000,2463.75000,1.0,197 days 18:38:45
50%,22647.000000,4926.50000,1.0,615 days 14:55:00
75%,30035.500000,7389.25000,1.0,1144 days 09:13:45
max,33413.000000,9852.00000,1.0,5206 days 07:40:00


### set tartget asset list

In [ ]:
import math

def get_historical_counts(start_date, end_date, interval):

    # Maximum period per request (416 days)
    if interval != 'daily' and interval != 'hourly':
      print('set interval as hourly or daily')
      return
    delta = timedelta(hours=100) if interval == 'hourly' else timedelta(days=100)

    # Convert start and end date strings (with milliseconds) to datetime objects
    start = datetime.strptime(start_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    end = datetime.strptime(end_date, '%Y-%m-%dT%H:%M:%S.%fZ')

    no = math.ceil((end-start)/delta)
    return no

def estimate_counts(target_crypto_assets, interval):
  # Collect historical data for each asset
  failed_assets = []
  total_assets = len(target_crypto_assets)
  counts = 0
  exclusion = 0
  min_period = timedelta(days=-1)
  filtered_id = pd.read_excel('data/filtered_crypto_tracked_assets.xlsx')['id']

  for index, row in target_crypto_assets.iterrows():

      first_date = row['first_historical_data']
      last_date = row['last_historical_data']

      counts = counts + get_historical_counts(first_date, last_date, interval)

  return counts

In [ ]:
assets = os.listdir('data/crypto_ohlcv_hourly')

In [ ]:
filtered_assets = filtered_assets[~filtered_assets['UCID'].isin(df_crypto_active_assets['id'])]

In [ ]:
missing = []
for i in filtered_assets['UCID']:
  s = True
  for f in assets:
    if f.startswith(f'{i}_') & f.endswith(f'_25.json'):
      s = False
      break
  if s:
    missing.append(i)

len(missing)

1

In [ ]:
filtered_assets = filtered_assets[filtered_assets['UCID'].isin(missing)]
filtered_assets = filtered_assets.rename(columns={'UCID':'id', 'CMC_ticker':'symbol'})
filtered_assets

,N,tokenid,total_votes,link,closed_votes,proposalsCount,no_tl,id,id_note,CMC_link,symbol,CMC_Note,Name_Defillama,Symbol_Defillama,DefiIlama_link,Defillama_name_eth,Defillama_Note,_merge
228,229,beanstalkdao.eth,6662.0,https://snapshot.org/#/beanstalkdao.eth,6662.0,33.0,NaN,12503,NaN,https://coinmarketcap.com/currencies/beanstalk/,NaN,NaN,Beanstalk,BEAN,https://defillama.com/protocol/beanstalk,NaN,NaN,1
259,260,beanstalkfarms.eth,5301.0,https://snapshot.org/#/beanstalkfarms.eth,5301.0,149.0,NaN,12503,NaN,https://coinmarketcap.com/currencies/beanstalk/,NaN,NaN,Beanstalk,BEAN,https://defillama.com/protocol/beanstalk,NaN,Derivative of Beanstalk,1
364,365,acryptos,3189.0,https://snapshot.org/#/acryptos,2914.0,130.0,8.0,7844,NaN,https://coinmarketcap.com/currencies/acryptos/,ACS,NaN,ACryptoS,ACS,https://defillama.com/protocol/acryptos,ACryptoS (ACS),NaN,3
374,375,galaxygoggle.eth,3056.0,https://snapshot.org/#/galaxygoggle.eth,3056.0,1.0,130.0,15268,NaN,https://coinmarketcap.com/currencies/galaxygog...,GG,NaN,GalaxyGoogle DAO,GG,https://defillama.com/protocol/galaxygoogle-dao,NaN,NaN,3
382,383,elasticdao.eth,2925.0,https://snapshot.org/#/elasticdao.eth,2925.0,15.0,107.0,9154,NaN,https://coinmarketcap.com/currencies/elastic-g...,EGT,NaN,ElasticSwap,TIC,https://defillama.com/protocol/elasticswap,NaN,NaN,3
384,385,fortressdao.eth,2900.0,https://snapshot.org/#/fortressdao.eth,2900.0,22.0,124.0,14861,NaN,https://coinmarketcap.com/currencies/fortressdao/,FORT,NaN,Fortress,FORT,https://defillama.com/protocol/fortress,NaN,NaN,3
388,389,saynotorug.eth,2871.0,https://snapshot.org/#/saynotorug.eth,2871.0,35.0,283.0,14971,NaN,https://coinmarketcap.com/currencies/r-u-gener...,RUG,NaN,R U Generous,RUG,https://defillama.com/protocol/r-u-generous,NaN,NaN,3
405,406,saddlefinance.eth,2664.0,https://snapshot.org/#/saddlefinance.eth,2664.0,88.0,281.0,14791,NaN,https://coinmarketcap.com/currencies/saddle-fi...,SDL,NaN,Saddle Finance,SDL,https://defillama.com/protocol/saddle-finance,Saddle Finance (SDL),NaN,3
415,416,sharedstake.eth,2587.0,https://snapshot.org/#/sharedstake.eth,2587.0,53.0,288.0,8445,NaN,https://coinmarketcap.com/currencies/sharedstake/,SGT,NaN,SharedStake,SGT,https://defillama.com/protocol/sharedstake,SharedStake (SGT),NaN,3
431,432,lobis.eth,2502.0,https://snapshot.org/#/lobis.eth,2502.0,25.0,184.0,15093,NaN,https://coinmarketcap.com/currencies/lobis/,LOBI,NaN,Lobis,LOBI,https://defillama.com/protocol/lobis,NaN,NaN,3


In [ ]:
filtered_assets.loc[filtered_assets['symbol'].isna(), 'symbol']='NaN'

***
# Collect Asset prices

In [ ]:
# Convert date to Unix timestamps for API
def date_to_unix(date_str):
    return int(time.mktime(time.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%fZ')))

In [ ]:
def get_historical_daily(session, output_dir, asset_id, start_date, end_date, retries=10):
    historical_url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'

    params = {
        'id': asset_id,
        'time_start': start_date,  # Unix timestamp for start
        'time_end': end_date,      # Unix timestamp for end
        'interval': 'daily',       # Daily data
        'count':10000
    }

    attempt = 0
    success = False
    while attempt < retries:
        try:
            response = session.get(historical_url, params=params, timeout=60)
            if response.status_code == 200:
                file_name = f"{output_dir}{asset_id}.json"
                success = True
                with open(file_name, "wb") as file:
                    file.write(response.content)
                return success
            else:
                print(f"Error {response.status_code} for asset {asset_id}, retrying...")
                attempt += 1
                time.sleep(60)  # Small delay before retry
        except Exception as e:
            print(f"Exception: {e}, retrying...")
            attempt += 1
            time.sleep(60)

    print(f"Failed to fetch data for asset {asset_id} after {retries} attempts.")
    return success

In [ ]:
# Create directory to store CSV files if it doesn't exist
output_dir = 'data/crypto_ohlcv_daily_2/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Collect historical data for each asset
failed_assets = []
total_assets = len(target_crypto_assets)

minimum_start_time = '2010-01-01T00:00:00.000Z'
minimum_start_timestamp = date_to_unix(minimum_start_time)
maximum_start_time = '2019-10-09T04:00:00.000Z' #'2024-10-09T04:00:00.000Z'
maximum_start_timestamp = date_to_unix(maximum_start_time)

first_date = minimum_start_timestamp # if not given
last_date = maximum_start_timestamp # if not given

session = Session()
session.headers.update(headers)

for index, row in target_crypto_assets.iterrows():
    asset_id = row['id']
    #first_date = row['first_historical_data']
    #last_date = row['last_historical_data']

    # Progress tracking
    progress = (index + 1) / total_assets * 100
    print(f"Progress: {progress:.2f}%, Collecting ID: {asset_id}")

    # If the first_date is newer than the minimum allowed start time, fix it
    #if start_date < minimum_start_timestamp:
    #    print(f"Fixing start date for asset {asset_id} ({symbol}) to {minimum_start_time}")
    #    start_date = minimum_start_timestamp
    #    first_date = real_collected_start_time  # Update first_date for naming purposes
    # Fetch historical data with retry
    success = get_historical_daily(session, output_dir, asset_id, first_date, last_date)

    with open('log/proccessed_daily_ohlcv.txt', 'a') as f:
        f.write(f"{row['id']}:{success}\n")

print("Data collection completed. Check failed_assets.txt for any failed assets.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Progress: 45.41%, Collecting ID: 8423
Progress: 45.42%, Collecting ID: 8428
Progress: 45.43%, Collecting ID: 8429
Progress: 45.44%, Collecting ID: 8430
Progress: 45.45%, Collecting ID: 8431
Progress: 45.46%, Collecting ID: 8432
Progress: 45.47%, Collecting ID: 8433
Progress: 45.48%, Collecting ID: 8434
Progress: 45.49%, Collecting ID: 8435
Progress: 45.50%, Collecting ID: 8436
Progress: 45.51%, Collecting ID: 8437
Progress: 45.52%, Collecting ID: 8439
Progress: 45.53%, Collecting ID: 8440
Progress: 45.54%, Collecting ID: 8441
Progress: 45.55%, Collecting ID: 8445
Progress: 45.56%, Collecting ID: 8446
Progress: 45.57%, Collecting ID: 8447
Progress: 45.58%, Collecting ID: 8450
Progress: 45.59%, Collecting ID: 8451
Progress: 45.60%, Collecting ID: 8453
Progress: 45.61%, Collecting ID: 8454
Progress: 45.62%, Collecting ID: 8455
Error 429 for asset 8455, retrying...
Progress: 45.63%, Collecting ID: 8456
Progress: 45.64%, Collecting ID: 8457
Progress: 45.6

In [ ]:
false_ids = []

# Open and read the log file
with open('log/proccessed_dailt_ohlcv.txt', 'r') as f:
    for line in f:
        # Use regex to check if the line ends with ':False'
        match = re.match(r"(\d+):False", line.strip())
        if match:
            # Extract the ID and add it to the list
            false_ids.append(int(match.group(1)))

print("False asset IDs:", false_ids[5:])

False asset IDs: []


In [ ]:
target_crypto_assets = df_crypto_inactive_assets

In [ ]:
output_dir = 'data/crypto_ohlcv_daily/'
if not os.path.exists(output_dir):
      os.makedirs(output_dir)

with open('log/proccessed_daily_ohlcv.txt', 'w') as f:
    pass

# Create a queue and add all rows to the queue
row_queue = queue.Queue()

for index, row in target_crypto_assets.iterrows():
    row_queue.put(row)

total_tasks = row_queue.qsize()
print(f"Queue uploaded. # of total tasks: {total_tasks}")

minimum_start_time = '2010-01-01T00:00:00.000Z'
minimum_start_timestamp = date_to_unix(minimum_start_time)
maximum_start_time = '2019-10-09T04:00:00.000Z' #'2024-10-09T04:00:00.000Z'
maximum_start_timestamp = date_to_unix(maximum_start_time)

first_date = minimum_start_timestamp # if not given
last_date = maximum_start_timestamp # if not given

stop_event = threading.Event()

def worker(worker_no, row_queue, total_tasks):
    print(f'Thread {worker_no} starting...')

    session = Session()
    session.headers.update(headers)
    while not stop_event.is_set():
        try:
            # Try to get the next row from the queue (non-blocking)
            print(f"  thread {worker_no}: requesting for a row")
            row = row_queue.get_nowait()

            print(f"  thread {worker_no}: row received. ID (id:{row['id']}) processing")

            # Call the pre-defined function for the current row
            success = get_historical_daily(session, output_dir, row['id'], first_date, last_date)
            with open('log/proccessed_dailt_ohlcv.txt', 'a') as f:
                f.write(f"{row['id']}:{success}\n")

            # Mark the task as done
            row_queue.task_done()
        except queue.Empty:
            break  # If the queue is empty, exit the loop

        # Compute and display progress
        remaining_tasks = row_queue.qsize()  # Remaining tasks in the queue
        completed_tasks = total_tasks - remaining_tasks
        progress = (completed_tasks / total_tasks) * 100
        print(f"Progress: {completed_tasks}/{total_tasks} tasks completed ({progress:.2f}%)")

# List to hold the threads
threads = []
num_workers = 8  # Number of worker threads

# Graceful shutdown handling
try:
    # Create and start worker threads
    for i in range(num_workers):
        thread = threading.Thread(target=worker, args=(i, row_queue, total_tasks))
        thread.start()
        threads.append(thread)

    print('All thread activated and working...')

    # Wait for all tasks to be completed
    row_queue.join()
    print("Complete")

except KeyboardInterrupt:
    print("Stopping all threads...")
    stop_event.set()  # Signal all threads to stop
    for thread in threads:
        thread.join()  # Ensure all threads are finished
    print("All threads stopped.")

Exception in thread Thread-12 (worker):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13 (worker):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-29-0a6881cd182d>", line 38, in worker
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
NameError: name 'first_date' is not defined
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-29-0a6881cd182d>", line 38, in worker
Exception in thread Thread-14 (worker):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2

Queue uploaded. # of total tasks: 1754
Thread 0 starting...
  thread 0: requesting for a row
  thread 0: row received. ID (id:7) processing
Thread 1 starting...
  thread 1: requesting for a row
  thread 1: row received. ID (id:12) processing
Thread 2 starting...Thread 3 starting...
  thread 2: requesting for a row
  thread 2: row received. ID (id:15) processing

  thread 3: requesting for a row
  thread 3: row received. ID (id:17) processing
Thread 4 starting...
  thread 4: requesting for a row
  thread 4: row received. ID (id:19) processing
Thread 5 starting...Thread 6 starting...
  thread 6: requesting for a row
  thread 6: row received. ID (id:20) processing
  thread 5: requesting for a row
  thread 5: row received. ID (id:22) processing

Thread 7 starting...
All thread activated and working...
  thread 7: requesting for a row
  thread 7: row received. ID (id:23) processing
Stopping all threads...
All threads stopped.


In [ ]:
max_id = 28683  # Initialize max_id

with open('log/proccessed_dailt_ohlcv.txt', 'r') as f:
    for line in f:
        # Strip newline and split by ":"
        parts = line.strip().split(":")

        # Ensure the line has the correct format
        if len(parts) == 2:
            asset_id = int(parts[0])  # Convert id to integer

            # Update max_id if this asset_id is greater
            if max_id is None or asset_id > max_id:
                max_id = asset_id

print("Maximum ID:", max_id)

target_crypto_assets = target_crypto_assets[target_crypto_assets['id']>max_id]
target_crypto_assets

Maximum ID: 28683


,id,rank,name,symbol,slug,is_active,platform
1752,29172,NaN,MYRO DRAGON,MYRODRAGON,myro-dragon,0,"{'id': 14, 'name': 'BNB Smart Chain (BEP20)', ..."
1753,33220,NaN,[deprecated]Major,[deprecated]MAJOR,deprecated-majoroftelegram,0,NaN


In [ ]:
import os
import re

def get_asset_ids_with_incorrect_max_date(directory_path, expected_max_date="2024-10-13"):
    # Regex to extract asset_id, symbol, min_date, and max_date from filenames
    pattern = re.compile(r"(\d+)_(\w+)_(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})\.csv")

    incorrect_assets = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            match = pattern.match(filename)
            if match:
                asset_id, symbol, min_date, max_date = match.groups()
                if max_date != expected_max_date:
                    incorrect_assets.append(asset_id)

    return incorrect_assets

# Example usage
directory_path = 'crypto/ohlcv_hourly'
incorrect_asset_ids = get_asset_ids_with_incorrect_max_date(directory_path)
print(len(incorrect_asset_ids), incorrect_asset_ids)

0 []


In [ ]:
os.listdir(directory_path)

['29169_QUBIC_1.json',
 '29175_WEN_1.json',
 '29191_MXM_1.json',
 '29166_BULL_2.json',
 '29188_CNG_1.json',
 '29169_QUBIC_2.json',
 '29175_WEN_2.json',
 '29191_MXM_2.json',
 '29197_ETHI_1.json',
 '29188_CNG_2.json',
 '29197_ETHI_2.json',
 '29194_LYRA_1.json',
 '29186_PEAS_1.json',
 '29209_BOOM_1.json',
 '29210_JUP_1.json',
 '29186_PEAS_2.json',
 '29211_EGG_1.json',
 '29209_BOOM_2.json',
 '29194_LYRA_2.json',
 '29210_JUP_2.json',
 '29211_EGG_2.json',
 '29212_MCN_1.json',
 '29219_TRUMP_1.json',
 '29219_TRUMP_2.json',
 '29221_LNR_1.json',
 '29229_PROS_1.json',
 '29212_MCN_2.json',
 '29231_ANUS_1.json',
 '29234_LION_1.json',
 '29221_LNR_2.json',
 '29229_PROS_2.json',
 '29231_ANUS_2.json',
 '29234_LION_2.json',
 '29223_JAN_1.json',
 '29223_JAN_2.json',
 '29200_DEFI_1.json',
 '29200_DEFI_2.json',
 '29235_FIRE_1.json',
 '29241_GME_1.json',
 '29254_MEOW_1.json',
 '29253_HOLD_1.json',
 '29265_INS_1.json',
 '29235_FIRE_2.json',
 '29241_GME_2.json',
 '29253_HOLD_2.json',
 '29254_MEOW_2.json',
 '2

In [ ]:
import os
import re

def delete_files_with_ids(directory_path, id_list):
    deleted_files = []

    for filename in os.listdir(directory_path):
        file_id = filename.split('_')[0]  # Get the first part before '_'
        if file_id in id_list:
            file_path = os.path.join(directory_path, filename)
            os.remove(file_path)  # Delete the file
            deleted_files.append(filename)
            print(f"Deleted file: {filename}")

    if not deleted_files:
        print("No matching files found to delete.")
    else:
        print(f"Deleted {len(deleted_files)} files matching the IDs in the list.")

# Example usage
directory_path = 'crypto/ohlcv_hourly'
delete_files_with_ids(directory_path, incorrect_asset_ids)

Deleted file: 12156_ASIA_2021-09-26_2024-08-03.csv
Deleted file: 12182_BLT_2021-09-29_2022-11-20.csv
Deleted file: 12198_BOSS_2021-09-29_2024-08-06.csv
Deleted file: 12208_TXT_2021-09-30_2022-11-20.csv
Deleted file: 12214_VERSE_2021-09-30_2022-11-21.csv
Deleted file: 12221_RPG_2021-10-07_2022-05-04.csv
Deleted file: 12209_AIR_2022-02-10_2023-10-28.csv
Deleted file: 4983_DMTC_2019-11-29_2021-01-18.csv
Deleted file: 4974_EXM_2019-12-04_2023-05-05.csv
Deleted file: 4956_MAPO_2020-01-06_2023-06-07.csv
Deleted file: 5002_SCAP_2019-12-05_2022-03-16.csv
Deleted file: 4957_BIP_2020-04-22_2022-08-02.csv
Deleted file: 5005_ARX_2020-02-02_2024-08-23.csv
Deleted 13 files matching the IDs in the list.


In [ ]:
 target_crypto_assets = filtered_assets
 target_crypto_assets

,N,tokenid,total_votes,link,closed_votes,proposalsCount,no_tl,id,id_note,CMC_link,symbol,CMC_Note,Name_Defillama,Symbol_Defillama,DefiIlama_link,Defillama_name_eth,Defillama_Note,_merge
228,229,beanstalkdao.eth,6662.0,https://snapshot.org/#/beanstalkdao.eth,6662.0,33.0,NaN,12503,NaN,https://coinmarketcap.com/currencies/beanstalk/,NaN,NaN,Beanstalk,BEAN,https://defillama.com/protocol/beanstalk,NaN,NaN,1
259,260,beanstalkfarms.eth,5301.0,https://snapshot.org/#/beanstalkfarms.eth,5301.0,149.0,NaN,12503,NaN,https://coinmarketcap.com/currencies/beanstalk/,NaN,NaN,Beanstalk,BEAN,https://defillama.com/protocol/beanstalk,NaN,Derivative of Beanstalk,1
364,365,acryptos,3189.0,https://snapshot.org/#/acryptos,2914.0,130.0,8.0,7844,NaN,https://coinmarketcap.com/currencies/acryptos/,ACS,NaN,ACryptoS,ACS,https://defillama.com/protocol/acryptos,ACryptoS (ACS),NaN,3
374,375,galaxygoggle.eth,3056.0,https://snapshot.org/#/galaxygoggle.eth,3056.0,1.0,130.0,15268,NaN,https://coinmarketcap.com/currencies/galaxygog...,GG,NaN,GalaxyGoogle DAO,GG,https://defillama.com/protocol/galaxygoogle-dao,NaN,NaN,3
382,383,elasticdao.eth,2925.0,https://snapshot.org/#/elasticdao.eth,2925.0,15.0,107.0,9154,NaN,https://coinmarketcap.com/currencies/elastic-g...,EGT,NaN,ElasticSwap,TIC,https://defillama.com/protocol/elasticswap,NaN,NaN,3
384,385,fortressdao.eth,2900.0,https://snapshot.org/#/fortressdao.eth,2900.0,22.0,124.0,14861,NaN,https://coinmarketcap.com/currencies/fortressdao/,FORT,NaN,Fortress,FORT,https://defillama.com/protocol/fortress,NaN,NaN,3
388,389,saynotorug.eth,2871.0,https://snapshot.org/#/saynotorug.eth,2871.0,35.0,283.0,14971,NaN,https://coinmarketcap.com/currencies/r-u-gener...,RUG,NaN,R U Generous,RUG,https://defillama.com/protocol/r-u-generous,NaN,NaN,3
405,406,saddlefinance.eth,2664.0,https://snapshot.org/#/saddlefinance.eth,2664.0,88.0,281.0,14791,NaN,https://coinmarketcap.com/currencies/saddle-fi...,SDL,NaN,Saddle Finance,SDL,https://defillama.com/protocol/saddle-finance,Saddle Finance (SDL),NaN,3
415,416,sharedstake.eth,2587.0,https://snapshot.org/#/sharedstake.eth,2587.0,53.0,288.0,8445,NaN,https://coinmarketcap.com/currencies/sharedstake/,SGT,NaN,SharedStake,SGT,https://defillama.com/protocol/sharedstake,SharedStake (SGT),NaN,3
431,432,lobis.eth,2502.0,https://snapshot.org/#/lobis.eth,2502.0,25.0,184.0,15093,NaN,https://coinmarketcap.com/currencies/lobis/,LOBI,NaN,Lobis,LOBI,https://defillama.com/protocol/lobis,NaN,NaN,3


In [ ]:
log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

import re

# Function to clean the symbol
def clean_symbol(symbol):
    return re.sub(r'[^a-zA-Z0-9]', '', symbol)

# Function to log failed attempts
def log_failure(start, end, error_message):
    with open(os.path.join(log_dir, 'fail_log_hourly.txt'), 'a') as log_file:
        log_file.write(f"Failed to fetch data from {start} to {end} - Error: {error_message}\n")

def get_historical_hourly(session, asset_id, symbol, start_date, end_date, max_attempts=3, output_dir='data/crypto_ohlcv_hourly/'):
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'

    # Maximum period per request (416 days)
    max_period = timedelta(hours=5000)

    # Convert start and end date strings (with milliseconds) to datetime objects
    if start_date:
        start = datetime.strptime(start_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    else:
        start = datetime.strptime('2010-07-13T00:05:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')
    if end_date:
        end = datetime.strptime(end_date, '%Y-%m-%dT%H:%M:%S.%fZ')
    else:
        end = datetime.strptime('2024-10-13', '%Y-%m-%d')

    no = 1
    while start < end:
        attempts = 0
        # Calculate the current end date for this request
        current_end = min(start + max_period, end)

        # Format the start and end time in ISO 8601 format without milliseconds
        params = {
            'id': asset_id,  # Use asset_id instead of symbol
            'interval': 'hourly',
            'time_start': start.strftime('%Y-%m-%dT%H:%M:%S'),  # Drop the milliseconds here
            'time_end': current_end.strftime('%Y-%m-%dT%H:%M:%S'),
            'time_period':'hourly',
            'count': 10000
        }

        # Retry logic with max_attempts
        success = False
        while attempts < max_attempts and not success:
            attempts += 1
            try:
                # Send the API request
                response = session.get(url, params=params, timeout=120)

                if response.status_code == 200:
                    #data = response.json()['data']  # Directly get the data
                    file_name = f"{output_dir}{asset_id}_{clean_symbol(symbol)}_{no}.json"
                    with open(file_name, "wb") as file:
                        file.write(response.content)
                    #with open(file_name, 'wb') as file:
                    #    pickle.dump(data, file)
                    success = True
                    no = no + 1
                else:
                    time.sleep(60)
                    print(f"{asset_id}_{symbol}_{no}: Attempt {attempts} failed with status code {response.status_code} - {response.text}")
                    if attempts == max_attempts:
                        log_failure(asset_id, start.strftime('%Y-%m-%dT%H:%M:%S'), current_end.strftime('%Y-%m-%dT%H:%M:%S'), response.text)
            except Exception as e:
                time.sleep(60)
                print(f"{asset_id}_{symbol}_{no}: Attempt {attempts} failed due to an exception: {e}")
                if attempts == max_attempts:
                    log_failure(asset_id, start.strftime('%Y-%m-%dT%H:%M:%S'), current_end.strftime('%Y-%m-%dT%H:%M:%S'), str(e))
                    return

        # Update the start time for the next request
        start = current_end

In [ ]:
target_crypto_assets = df_crypto_untracked_assets[df_crypto_untracked_assets['id']>=4932]

In [ ]:
target_crypto_assets = df_crypto_untracked_assets[df_crypto_untracked_assets['id'].isin(missing)]

In [ ]:
target_crypto_assets = filtered_assets
target_crypto_assets

,N,tokenid,total_votes,link,closed_votes,proposalsCount,no_tl,id,id_note,CMC_link,symbol,CMC_Note,Name_Defillama,Symbol_Defillama,DefiIlama_link,Defillama_name_eth,Defillama_Note,_merge
4,5,linea-build.eth,3135558.0,https://snapshot.org/#/linea-build.eth,3135558.0,2.0,NaN,NaN,NaN,https://coinmarketcap.com/currencies/linea/,NaN,NaN,Linea,NaN,https://defillama.com/chain/Linea,NaN,NaN,1
18,19,shellprotocol.eth,177117.0,https://snapshot.org/#/shellprotocol.eth,177117.0,55.0,289.0,7499,no price,https://coinmarketcap.com/currencies/shell-pro...,SHELL,NaN,Shell Protocol,SHELL,https://defillama.com/protocol/shell-protocol,NaN,NaN,3
607,608,legendofarcadia.eth,1474.0,https://snapshot.org/#/legendofarcadia.eth,1474.0,6.0,NaN,32273,NaN,https://coinmarketcap.com/currencies/legend-of...,ARCA,NaN,Arcadia Finance,NaN,https://defillama.com/protocol/arcadia-finance,NaN,NaN,1
662,663,themis-labs.eth,1293.0,https://snapshot.org/#/themis-labs.eth,1293.0,3.0,NaN,25914,Price not tracked,https://coinmarketcap.com/currencies/themis-pr...,THS,NaN,Themis Protocol,TMS,https://defillama.com/protocol/themis-protocol,NaN,NaN,1
703,704,solvgov.eth,1179.0,https://snapshot.org/#/solvgov.eth,1179.0,4.0,NaN,NaN,price not tracked,https://coinmarketcap.com/currencies/solv-prot...,NaN,NaN,Solv Protocol,NaN,https://defillama.com/protocol/solv-protocol,NaN,NaN,1


In [ ]:
stop_event = threading.Event()

def worker(worker_no, row_queue, total_tasks):
    print(f'Thread {worker_no} starting...')

    session = Session()
    session.headers.update(headers)
    while not stop_event.is_set():
        try:
            # Try to get the next row from the queue (non-blocking)
            print(f"  thread {worker_no}: requesting for a row")
            row = row_queue.get_nowait()

            print(f"  thread {worker_no}: row received. ID (id:{row['id']}) processing")

            # Call the pre-defined function for the current row
            get_historical_hourly(session, row['id'], row['symbol'], row.get('first_historical_data', None), row.get('last_historical_data', None))

            # Mark the task as done
            row_queue.task_done()

            with open('log/proccessed.txt', 'a') as f:
                f.write(f"{row['id']}\n")
        except queue.Empty:
            break  # If the queue is empty, exit the loop

        # Compute and display progress
        remaining_tasks = row_queue.qsize()  # Remaining tasks in the queue
        completed_tasks = total_tasks - remaining_tasks
        progress = (completed_tasks / total_tasks) * 100
        print(f"Progress: {completed_tasks}/{total_tasks} tasks completed ({progress:.2f}%)")

# Create a queue and add all rows to the queue
row_queue = queue.Queue()

for index, row in target_crypto_assets.iterrows():
    row_queue.put(row)

total_tasks = row_queue.qsize()
print(f"Queue uploaded. # of total tasks: {total_tasks}")

# List to hold the threads
threads = []
num_workers = 8  # Number of worker threads

# Graceful shutdown handling
try:
    # Create and start worker threads
    for i in range(num_workers):
        thread = threading.Thread(target=worker, args=(i, row_queue, total_tasks))
        thread.start()
        threads.append(thread)

    print('All thread activated and working...')

    # Wait for all tasks to be completed
    row_queue.join()

except KeyboardInterrupt:
    print("Stopping all threads...")
    stop_event.set()  # Signal all threads to stop
    for thread in threads:
        thread.join()  # Ensure all threads are finished
    print("All threads stopped.")

Queue uploaded. # of total tasks: 5
Thread 0 starting...Thread 1 starting...

  thread 0: requesting for a row
  thread 0: row received. ID (id:NaN) processing
  thread 1: requesting for a rowThread 2 starting...
  thread 2: requesting for a row
  thread 2: row received. ID (id:7499) processing

Thread 3 starting...
  thread 3: requesting for a row
  thread 3: row received. ID (id:32273) processing
  thread 1: row received. ID (id:25914) processing
Thread 4 starting...
  thread 4: requesting for a row
  thread 4: row received. ID (id:NaN) processing
Thread 5 starting...
Thread 6 starting...
  thread 6: requesting for a row
  thread 5: requesting for a rowThread 7 starting...

  thread 7: requesting for a row
All thread activated and working...
NaN_nan_1: Attempt 1 failed with status code 400 - {"status":{"timestamp":"2024-12-27T10:48:12.827Z","error_code":400,"error_message":"\"id\" should only include comma-separated numeric CoinMarketCap cryptocurrency ids","elapsed":0,"credit_count"

Exception in thread Thread-35 (worker):
Traceback (most recent call last):
  File "<ipython-input-29-37ee8d0cfb1c>", line 69, in get_historical_hourly
TypeError: log_failure() takes 3 positional arguments but 4 were given

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-76-c43c34321486>", line 17, in worker
  File "<ipython-input-29-37ee8d0cfb1c>", line 74, in get_historical_hourly
TypeError: log_failure() takes 3 positional arguments but 4 were given
Exception in thread Thread-39 (worker):
Traceback (most recent call last):
  File "<ipython-input-29-37ee8d0cfb1c>", line 69, in get_historical_hourly
TypeError: log_failure() takes 3 positional arguments but 4 were given

During handling of the above exception, another

NaN_nan_1: Attempt 3 failed due to an exception: log_failure() takes 3 positional arguments but 4 were given
NaN_nan_1: Attempt 3 failed due to an exception: log_failure() takes 3 positional arguments but 4 were given
Stopping all threads...
All threads stopped.


In [ ]:
stop_event = threading.Event()

def worker(worker_no, row_queue, total_tasks):
    print(f'Thread {worker_no} starting...')

    session = Session()
    session.headers.update(headers)
    while not stop_event.is_set():
        try:
            # Try to get the next row from the queue (non-blocking)
            print(f"  thread {worker_no}: requesting for a row")
            id = id_queue.get_nowait()

            print(f"  thread {worker_no}: row received. ID (id:{id}) processing")

            # Call the pre-defined function for the current row
            get_historical_hourly(session, id, row['symbol'], None, None)

            # Mark the task as done
            row_queue.task_done()

            with open('log/proccessed.txt', 'a') as f:
                f.write(f"{row['id']}\n")
        except queue.Empty:
            break  # If the queue is empty, exit the loop

        # Compute and display progress
        remaining_tasks = row_queue.qsize()  # Remaining tasks in the queue
        completed_tasks = total_tasks - remaining_tasks
        progress = (completed_tasks / total_tasks) * 100
        print(f"Progress: {completed_tasks}/{total_tasks} tasks completed ({progress:.2f}%)")

# Create a queue and add all rows to the queue
id_queue = queue.Queue()

for index, row in target_crypto_ids.iterrows():
    id_queue.put(row)

total_tasks = row_queue.qsize()
print(f"Queue uploaded. # of total tasks: {total_tasks}")

# List to hold the threads
threads = []
num_workers = 8  # Number of worker threads

# Graceful shutdown handling
try:
    # Create and start worker threads
    for i in range(num_workers):
        thread = threading.Thread(target=worker, args=(i, row_queue, total_tasks))
        thread.start()
        threads.append(thread)

    print('All thread activated and working...')

    # Wait for all tasks to be completed
    row_queue.join()

except KeyboardInterrupt:
    print("Stopping all threads...")
    stop_event.set()  # Signal all threads to stop
    for thread in threads:
        thread.join()  # Ensure all threads are finished
    print("All threads stopped.")

***
# Aggregate Data

In [ ]:
file_path = 'data/crypto_ohlcv_daily/3639.json'
with open(file_path, 'r') as f:
    data = json.load(f)

quotes = data["data"]["quotes"]

flattened_data = []
for entry in quotes:
    quote_data = entry["quote"]["USD"]
    flat_entry = {
        "time_open": entry["time_open"],
        "time_close": entry["time_close"],
        "time_high": entry["time_high"],
        "time_low": entry["time_low"],
        "open": quote_data["open"],
        "high": quote_data["high"],
        "low": quote_data["low"],
        "close": quote_data["close"],
        "volume": quote_data["volume"],
        "market_cap": quote_data["market_cap"]
    }
    flattened_data.append(flat_entry)

df = pd.DataFrame(flattened_data)
df

,time_open,time_close,time_high,time_low,open,high,low,close,volume,market_cap
0,2019-10-10T00:00:00.000Z,2019-10-10T23:59:59.999Z,2019-10-10T14:35:13.000Z,2019-10-10T09:24:13.000Z,0.000469,0.000698,0.000420,0.000442,16590.57,397151.202641
1,2019-10-11T00:00:00.000Z,2019-10-11T23:59:59.999Z,2019-10-11T03:28:09.000Z,2019-10-11T11:03:13.000Z,0.000445,0.000502,0.000393,0.000420,17794.74,378030.443040
2,2019-10-12T00:00:00.000Z,2019-10-12T23:59:59.999Z,2019-10-12T23:02:09.000Z,2019-10-12T02:20:12.000Z,0.000420,0.000471,0.000389,0.000443,15205.52,397940.016187
3,2019-10-13T00:00:00.000Z,2019-10-13T23:59:59.999Z,2019-10-13T00:18:05.000Z,2019-10-13T08:56:14.000Z,0.000440,0.000503,0.000389,0.000408,11287.29,367160.444447
4,2019-10-14T00:00:00.000Z,2019-10-14T23:59:59.999Z,2019-10-14T03:04:06.000Z,2019-10-14T00:23:06.000Z,0.000408,0.000429,0.000397,0.000411,11708.02,369275.357073
...,...,...,...,...,...,...,...,...,...,...
1822,2024-10-05T00:00:00.000Z,2024-10-05T23:59:59.999Z,2024-10-05T00:51:00.000Z,2024-10-05T00:51:00.000Z,0.000017,0.000017,0.000017,0.000017,0.00,0.000000
1823,2024-10-06T00:00:00.000Z,2024-10-06T23:59:59.999Z,2024-10-06T00:54:00.000Z,2024-10-06T00:54:00.000Z,0.000017,0.000017,0.000017,0.000017,0.00,0.000000
1824,2024-10-07T00:00:00.000Z,2024-10-07T23:59:59.999Z,2024-10-07T00:57:00.000Z,2024-10-07T00:57:00.000Z,0.000017,0.000017,0.000017,0.000017,0.00,0.000000
1825,2024-10-08T00:00:00.000Z,2024-10-08T23:59:59.999Z,2024-10-08T00:00:00.000Z,2024-10-08T00:00:00.000Z,0.000017,0.000017,0.000017,0.000017,0.00,0.000000


In [ ]:
import glob

def process_files(directory, output_file, active_status):
    # Define the file path pattern to match all CSV files in the given directory
    file_pattern = os.path.join(directory, '*.json')

    # Collect all file paths matching the pattern
    files = glob.glob(file_pattern)

    # Loop through each file path and process one by one
    for i, file_path in enumerate(files, 1):
        # Extract the file name to retrieve asset_id and symbol
        asset_id = os.path.basename(file_path).replace('.json', '')

        # Print the current file being processed
        print(f"Processing file {i}/{len(files)}: {asset_id}")

        # Read the CSV file in chunks using Pandas
        #chunk_iter = pd.read_csv(file_path, chunksize=100000)
        with open(file_path, 'r') as f:
            data = json.load(f)
        symbol = data['data']['symbol']

        data["data"]["quotes"]

        flattened_data = []
        for entry in quotes:
            quote_data = entry["quote"]["USD"]
            flat_entry = {
                "time_open": entry["time_open"],
                "time_close": entry["time_close"],
                "time_high": entry["time_high"],
                "time_low": entry["time_low"],
                "open": quote_data["open"],
                "high": quote_data["high"],
                "low": quote_data["low"],
                "close": quote_data["close"],
                "volume": quote_data["volume"],
                "market_cap": quote_data["market_cap"]
            }
            flattened_data.append(flat_entry)

        df = pd.DataFrame(flattened_data)

        df['asset_id'] = asset_id
        df['symbol'] = symbol
        df['active'] = active_status

        df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

        # Process each chunk
        '''
        for chunk in chunk_iter:
            chunk['asset_id'] = asset_id
            chunk['symbol'] = symbol
            chunk['active'] = active_status  # Add 'active' column with the given active_status

            # Append chunk to the output file in 'append' mode
            chunk.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
        '''

# Define directories for active and inactive data
#active_directory = 'crypto/ohlcv_active_assets'
#inactive_directory = 'crypto/ohclv_inactive_assets'
untracked_directory = 'data/crypto_ohlcv_daily'

# Output file path
output_file = 'crypto/crypto_ohlcv_daily_untracked.csv'

process_files(untracked_directory, output_file, active_status=False)

#process_files(inactive_directory, output_file, active_status=False)

Streaming output truncated to the last 5000 lines.
Processing file 5001/10000: 9149
Processing file 5002/10000: 9157
Processing file 5003/10000: 9159
Processing file 5004/10000: 9156
Processing file 5005/10000: 9161
Processing file 5006/10000: 9162
Processing file 5007/10000: 9160
Processing file 5008/10000: 9163
Processing file 5009/10000: 9165
Processing file 5010/10000: 9164
Processing file 5011/10000: 9170
Processing file 5012/10000: 9168
Processing file 5013/10000: 9174
Processing file 5014/10000: 9166
Processing file 5015/10000: 9171
Processing file 5016/10000: 9183
Processing file 5017/10000: 9167
Processing file 5018/10000: 9181
Processing file 5019/10000: 9178
Processing file 5020/10000: 9189
Processing file 5021/10000: 9185
Processing file 5022/10000: 9186
Processing file 5023/10000: 9190
Processing file 5024/10000: 9195
Processing file 5025/10000: 9182
Processing file 5026/10000: 9192
Processing file 5027/10000: 9187
Processing file 5028/10000: 9197
Processing file 5029/1000

In [ ]:
chunksize = 100000  # Set this depending on the size of your data and memory availability
existing_csv_file = 'crypto/crypto_ohlcv_daily.csv'
new_csv_file = 'crypto/crypto_ohlcv_daily_untracked.csv'
for chunk in pd.read_csv(new_csv_file, chunksize=chunksize):
    chunk.to_csv(existing_csv_file, mode='a', header=not os.path.exists(existing_csv_file), index=False)

In [ ]:
new_df = pd.read_csv('crypto/crypto_ohlcv_daily.csv')

new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25484469 entries, 0 to 25484468
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   time_open   object 
 1   time_close  object 
 2   time_high   object 
 3   time_low    object 
 4   open        float64
 5   high        float64
 6   low         float64
 7   close       float64
 8   volume      float64
 9   market_cap  float64
 10  asset_id    int64  
 11  symbol      object 
 12  active      bool   
dtypes: bool(1), float64(6), int64(1), object(5)
memory usage: 2.3+ GB


In [ ]:
import zipfile

# Specify the CSV file path
csv_file = 'crypto/crypto_ohlcv_daily.csv'  # Replace with your CSV file path
zip_file ='crypto/crypto_ohlcv_daily.zip'  # Desired ZIP file name

# Create a ZIP file and add the CSV file to it
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write(csv_file, os.path.basename(csv_file))

In [ ]:
file_path = 'data/crypto_ohlcv_hourly/4974_EXM_1.json'

# Load directly as pickle and assume it's only `data`
try:
  with open(file_path, 'rb') as f:
    a = json.load(f)
except UnicodeDecodeError:
  with open(file_path, 'rb') as f:
    asset_data = pickle.load(f)
pd.DataFrame(asset_data)

,id,name,symbol,quotes
0,1,Bitcoin,BTC,"{'time_open': '2010-07-13T01:00:00.000Z', 'tim..."
1,1,Bitcoin,BTC,"{'time_open': '2010-07-13T02:00:00.000Z', 'tim..."
2,1,Bitcoin,BTC,"{'time_open': '2010-07-13T03:00:00.000Z', 'tim..."
3,1,Bitcoin,BTC,"{'time_open': '2010-07-13T04:00:00.000Z', 'tim..."
4,1,Bitcoin,BTC,"{'time_open': '2010-07-13T05:00:00.000Z', 'tim..."
...,...,...,...,...
9979,1,Bitcoin,BTC,"{'time_open': '2011-09-01T20:00:00.000Z', 'tim..."
9980,1,Bitcoin,BTC,"{'time_open': '2011-09-01T21:00:00.000Z', 'tim..."
9981,1,Bitcoin,BTC,"{'time_open': '2011-09-01T22:00:00.000Z', 'tim..."
9982,1,Bitcoin,BTC,"{'time_open': '2011-09-01T23:00:00.000Z', 'tim..."


In [ ]:
target_id_list = list(filtered_assets['UCID'])

In [ ]:
files = glob.glob(os.path.join(output_dir, '*'))  # Get all files and directories in the directory

# Loop through the files and delete them
for file_path in files:
    if os.path.isfile(file_path):  # Ensure it's a file and not a subdirectory
        os.remove(file_path)  # Delete the file
        print(f"Deleted: {file_path}")
    elif os.path.isdir(file_path):  # Optionally delete subdirectories as well
        os.rmdir(file_path)  # Delete the directory (only if empty)
        print(f"Deleted directory: {file_path}")

Deleted: crypto/request_hourly_tao/aggregate_fail_list_tao.txt
Deleted: crypto/request_hourly_tao/400_KORE_2019-07-13_2019-08-03.csv
Deleted: crypto/request_hourly_tao/24_AMC_2020-09-11_2020-09-18.csv
Deleted: crypto/request_hourly_tao/12_BQC_2013-05-19_2017-11-23.csv
Deleted: crypto/request_hourly_tao/7_DVC_2013-05-19_2017-11-23.csv
Deleted: crypto/request_hourly_tao/813_BITSILVER_2015-02-25_2020-09-23.csv
Deleted: crypto/request_hourly_tao/666_AU_2014-10-09_2019-06-01.csv
Deleted: crypto/request_hourly_tao/837_XCO_2015-03-06_2024-05-28.csv
Deleted: crypto/request_hourly_tao/799_SMLY_2015-01-28_2024-02-09.csv
Deleted: crypto/request_hourly_tao/833_GRC_2015-02-28_2024-02-09.csv
Deleted: crypto/request_hourly_tao/914_SPHR_2015-05-02_2022-06-09.csv
Deleted: crypto/request_hourly_tao/911_AIB_2015-04-30_2023-06-13.csv
Deleted: crypto/request_hourly_tao/1063_BCY_2015-09-14_2024-04-29.csv
Deleted: crypto/request_hourly_tao/1089_DUO_2015-10-24_2022-01-23.csv
Deleted: crypto/request_hourly_tao

In [ ]:
import json
import csv
import glob
import os
import pickle
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from datetime import datetime

# Define the directory where JSON files are located
output_dir = 'crypto/ohlcv_hourly'  # replace with your directory path
log_file = os.path.join(output_dir, 'aggregate_fail_list.txt')  # path to log failed files
log_lock = Lock()  # Lock for thread-safe logging

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Clear log file at the start
with open(log_file, 'w') as lf:
    lf.write("Failed files list:\n")

# Get list of JSON files
json_files = glob.glob('data/crypto_ohlcv_hourly/*.json')

# Group files by asset_id
files_by_asset = defaultdict(list)
for file_path in json_files:
    filename = os.path.basename(file_path)
    asset_id = filename.split('_')[0]

    if True: #int(asset_id) in target_id_list
      files_by_asset[asset_id].append(file_path)

# Total asset_ids to process for tracking overall progress
total_assets = len(files_by_asset)
print(total_assets)
completed_assets = 0  # Counter to track completed asset processing

# Function to process files for a single asset_id
def process_asset_files(asset_id, file_paths):
    rows = []
    dates = []
    symbol = None

    for file_idx, file_path in enumerate(file_paths, start=1):
        data = None
        existing_files = glob.glob(os.path.join(output_dir, f"{asset_id}_*.csv"))
        if existing_files:
            print(f"Skipping {asset_id} - aggregated file already exists.")
            return asset_id

        # First attempt: read as JSON
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            asset_data = data.get("data", data)  # Handle case for Format 1 and Format 2

        # Second attempt: If JSON load fails, try loading as a pickle file
        except UnicodeDecodeError:
            try:
                with open(file_path, 'rb') as f:
                    # Load directly as pickle and assume it's only `data`
                    asset_data = pickle.load(f)
            except (pickle.UnpicklingError, EOFError, ValueError) as e:
                # Log the failed file in a thread-safe manner
                with log_lock:
                    with open(log_file, 'a') as lf:
                        lf.write(f"{asset_id} - Failed to load {file_path}: {e}\n")
                print(f"Failed to read {file_path} as JSON or pickle. Error: {e}")
                continue

        # Extract required data for each quote entry, excluding `quote_timestamp`
        for quote in asset_data.get("quotes", []):
            row = {
                "asset_id": asset_id,
                "name": asset_data.get("name", ""),
                "symbol": asset_data.get("symbol", ""),
                "time_open": quote.get("time_open", ""),
                "time_close": quote.get("time_close", ""),
                "time_high": quote.get("time_high", ""),
                "time_low": quote.get("time_low", ""),
                "open": quote.get("quote", {}).get("USD", {}).get("open", ""),
                "high": quote.get("quote", {}).get("USD", {}).get("high", ""),
                "low": quote.get("quote", {}).get("USD", {}).get("low", ""),
                "close": quote.get("quote", {}).get("USD", {}).get("close", ""),
                "volume": quote.get("quote", {}).get("USD", {}).get("volume", ""),
                "market_cap": quote.get("quote", {}).get("USD", {}).get("market_cap", "")
            }
            rows.append(row)
            dates.append(row["time_open"])  # Collect all open dates
            symbol = asset_data.get("symbol", symbol)  # Update symbol if not already set

    # Determine min and max date in 'YYYY-MM-DD' format
    if dates:
        min_date = min(dates).split("T")[0]
        max_date = max(dates).split("T")[0]
    else:
        min_date = max_date = "unknown"

    # Define output CSV file path with date range
    csv_file = os.path.join(output_dir, f"{asset_id}_{symbol}_{min_date}_{max_date}.csv")

    # Save aggregated data for this asset_id to a CSV file if there are rows
    if rows:
        with open(csv_file, 'w', newline='') as f:
            # Create a CSV writer with the fieldnames matching our data structure
            writer = csv.DictWriter(f, fieldnames=rows[0].keys())
            writer.writeheader()
            writer.writerows(rows)

        print(f"Aggregated and saved data for asset_id {asset_id} to CSV as {csv_file}")
    else:
        print(f"No data to save for asset_id {asset_id}. Skipping CSV creation.")

    # Return the asset_id to signal completion for progress tracking
    return asset_id

# Using ThreadPoolExecutor with 8 workers to process each asset_id in parallel
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_asset_files, asset_id, file_paths) for asset_id, file_paths in files_by_asset.items()]

    for future in as_completed(futures):
        asset_id = future.result()
        # Update completed_assets and print overall progress
        completed_assets += 1
        overall_progress = (completed_assets / total_assets) * 100
        print(f"Completed processing for asset_id: {asset_id} - Overall progress: {overall_progress:.2f}%")

print(f"Aggregation complete. See {log_file} for failed files.")

14933
No data to save for asset_id 8417. Skipping CSV creation.
Completed processing for asset_id: 8417 - Overall progress: 0.01%
Aggregated and saved data for asset_id 8418 to CSV as crypto/ohlcv_hourly/8418_CTASK_2021-02-09_2022-08-21.csv
Aggregated and saved data for asset_id 8415 to CSV as crypto/ohlcv_hourly/8415_MRC_2021-07-21_2022-09-23.csv
Completed processing for asset_id: 8415 - Overall progress: 0.01%
Completed processing for asset_id: 8418 - Overall progress: 0.02%
Aggregated and saved data for asset_id 8414 to CSV as crypto/ohlcv_hourly/8414_FAI_2021-03-01_2022-08-25.csv
Aggregated and saved data for asset_id 8413 to CSV as crypto/ohlcv_hourly/8413_RPT_2021-02-08_2022-08-16.csv
Completed processing for asset_id: 8414 - Overall progress: 0.03%
Completed processing for asset_id: 8413 - Overall progress: 0.03%
Aggregated and saved data for asset_id 8428 to CSV as crypto/ohlcv_hourly/8428_MSC_2021-02-14_2022-08-23.csv
Completed processing for asset_id: 8428 - Overall progress:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-129-33824fade9f9>", line 123, in <cell line: 120>
    for future in as_completed(futures):
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 245, in as_completed
    waiter.event.wait(wait_timeout)
  File "/usr/lib/python3.10/threading.py", line 607, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above except

In [ ]:
len(filtered_assets['UCID'])

391

In [ ]:
filtered_assets['UCID'].drop_duplicates()

,UCID
1,18934
2,11841
3,7278
4,27657
5,11840
...,...
1168,9421
1170,14636
1171,14131
1173,8716


In [ ]:
len(assets)

359

In [ ]:
assets = os.listdir(output_dir)
missing = []
for i in filtered_assets['UCID']:
  s = True
  for f in assets:
    if f.startswith(f'{i}_'):
      s = False
      break
  if s:
    missing.append(i)

missing

[27657, 7499, 32273, 25914, 12165, 10932]

In [ ]:
filtered_assets = filtered_assets[filtered_assets['UCID'].isin(missing)]

In [ ]:
filtered_assets = filtered_assets[filtered_assets['CMC_ticker']!='IF']

In [ ]:
filtered_assets.loc[filtered_assets['CMC_ticker'].isna(), 'UCID']='NaN'

In [ ]:
filtered_assets

,N,tokenid,total_votes,link,closed_votes,proposalsCount,no_tl,UCID,id_note,CMC_link,CMC_ticker,CMC_Note,Name_Defillama,Symbol_Defillama,DefiIlama_link,Defillama_name_eth,Defillama_Note,_merge
4,5,linea-build.eth,3135558.0,https://snapshot.org/#/linea-build.eth,3135558.0,2.0,NaN,NaN,NaN,https://coinmarketcap.com/currencies/linea/,NaN,NaN,Linea,NaN,https://defillama.com/chain/Linea,NaN,NaN,1
18,19,shellprotocol.eth,177117.0,https://snapshot.org/#/shellprotocol.eth,177117.0,55.0,289.0,7499,no price,https://coinmarketcap.com/currencies/shell-pro...,SHELL,NaN,Shell Protocol,SHELL,https://defillama.com/protocol/shell-protocol,NaN,NaN,3
607,608,legendofarcadia.eth,1474.0,https://snapshot.org/#/legendofarcadia.eth,1474.0,6.0,NaN,32273,NaN,https://coinmarketcap.com/currencies/legend-of...,ARCA,NaN,Arcadia Finance,NaN,https://defillama.com/protocol/arcadia-finance,NaN,NaN,1
662,663,themis-labs.eth,1293.0,https://snapshot.org/#/themis-labs.eth,1293.0,3.0,NaN,25914,Price not tracked,https://coinmarketcap.com/currencies/themis-pr...,THS,NaN,Themis Protocol,TMS,https://defillama.com/protocol/themis-protocol,NaN,NaN,1
703,704,solvgov.eth,1179.0,https://snapshot.org/#/solvgov.eth,1179.0,4.0,NaN,NaN,price not tracked,https://coinmarketcap.com/currencies/solv-prot...,NaN,NaN,Solv Protocol,NaN,https://defillama.com/protocol/solv-protocol,NaN,NaN,1


In [ ]:
import zipfile
import os

def zip_directory_exclude_log(directory_path, zip_name, exclude_file="aggregate_fail_list.txt"):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Traverse through all files and directories in the specified directory
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                # Skip the log file
                if file == exclude_file:
                    continue
                file_path = os.path.join(root, file)
                # Add each file to the zip, keeping the directory structure
                zipf.write(file_path, os.path.relpath(file_path, directory_path))
    print(f"{zip_name} created successfully, excluding the log file.")

# Example usage
directory_path = 'crypto/ohlcv_hourly'
zip_name = 'crypto/crypto_ohlcv_hourly_filtered_active.zip'
zip_directory_exclude_log(directory_path, zip_name)


***
# Save to DBX

In [ ]:
!pip install dropbox

In [ ]:
import dropbox
from dropbox.oauth import DropboxOAuth2FlowNoRedirect

dbx_app_secret, dbx_app_key = ['','']

with open('env/dbx_app_key.txt', 'r') as file:
    dbx_app_key = file.read()

with open('env/dbx_app_secret.txt', 'r') as file:
    dbx_app_secret = file.read()

auth_flow = DropboxOAuth2FlowNoRedirect(dbx_app_key, dbx_app_secret)

# Step 3: Get the authorization URL for the user
authorize_url = auth_flow.start()
print("1. Go to this URL to authorize the app:", authorize_url)
print("2. Click 'Allow' and copy the authorization code.")

# Step 4: User enters the authorization code
auth_code = input("Enter the authorization code here: ").strip()

# Step 5: Finish the OAuth process to get the access token
oauth_result = auth_flow.finish(auth_code)

# Store the access token
access_token = oauth_result.access_token
print(f"Access token: {access_token}")

# Step 6: Use the access token to connect to Dropbox API
dbx = dropbox.Dropbox(access_token)

# Example: List files in the root directory of the Dropbox account
for entry in dbx.files_list_folder('').entries:
    print(entry.name)


1. Go to this URL to authorize the app: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=9994p874eamilm2
2. Click 'Allow' and copy the authorization code.
Enter the authorization code here: DbNpKcUeVB0AAAAAAAAAP0lNVyPLBSv07MRAYbQym0g
Access token: sl.B_cYCrgMiwR3J_yC1NQZOze_-gg63kyCP4uxva0wbZJuPvj52uJWwbyBGhSGWgFvuMchoCxzZJ1pccVXnumt7f5k59HeVMeBYzwQ8iD-OzvIid2BHLJDyZm3YZiAWD3uggJvUZN17QF_tEy5_I3IJ88
Defi_rehypothecation
CMC_Token_Price DATA
KOR_BOND_DATA


In [ ]:
local_file_path = 'crypto/crypto_ohlcv_daily.zip'
dropbox_target_path = '/CMC_Token_Price DATA/crypto_ohlcv_daily.zip'

with open(local_file_path, 'rb') as f:
    dbx.files_upload(f.read(), dropbox_target_path)

print("File uploaded successfully to Dropbox!")

ApiError: ApiError('5c30afdaaab24c8abcfa714618a93ae6', UploadError('payload_too_large', None))

In [ ]:
def list_dropbox_folder(dropbox_path):
    """List all files and directories in a Dropbox folder."""
    try:
        result = dbx.files_list_folder(dropbox_path)
        items = {}

        # Collect all file and folder names from Dropbox
        for entry in result.entries:
            items[entry.name] = entry
        while result.has_more:
            result = dbx.files_list_folder_continue(result.cursor)
            for entry in result.entries:
                items[entry.name] = entry

        return items
    except dropbox.exceptions.ApiError as err:
        if isinstance(err.error, dropbox.files.ListFolderError) and err.error.is_path() and err.error.get_path().is_not_found():
            return {}  # Directory doesn't exist in Dropbox
        else:
            print(f"Error listing Dropbox folder {dropbox_path}: {err}")
            return {}

def upload_to_dropbox(local_path, dropbox_path):
    """Uploads files or directories to Dropbox, but only if they are missing."""

    # Get the list of existing files and directories in the Dropbox folder
    existing_items = list_dropbox_folder(dropbox_path)

    # If the directory doesn't exist in Dropbox, create it
    if not existing_items:
        try:
            dbx.files_create_folder_v2(dropbox_path)
            print(f'Created folder {dropbox_path} in Dropbox')
        except dropbox.exceptions.ApiError as err:
            print(f"Error creating folder {dropbox_path}: {err}")

    # Check if it's a directory and process files and subdirectories
    if os.path.isdir(local_path):
        # Iterate over the local directory content
        for item in os.listdir(local_path):
            item_local_path = os.path.join(local_path, item)
            item_dropbox_path = os.path.join(dropbox_path, item).replace("\\", "/")

            # Check if the item exists in Dropbox
            if item in existing_items:
                # If it's a directory in both local and Dropbox, recursively upload its contents
                if isinstance(existing_items[item], dropbox.files.FolderMetadata):
                    upload_to_dropbox(item_local_path, item_dropbox_path)
                else:
                    print(f'Skipping {item_local_path} as it already exists in Dropbox')
            else:
                # If it's a directory locally, recursively copy its contents
                if os.path.isdir(item_local_path):
                    upload_to_dropbox(item_local_path, item_dropbox_path)
                else:
                    # If it's a file, upload it to Dropbox
                    with open(item_local_path, "rb") as f:
                        dbx.files_upload(f.read(), item_dropbox_path)
                        print(f'Uploaded {item_local_path} to {item_dropbox_path}')

    # If it's a file, upload it only if it doesn't exist in Dropbox
    else:
        if os.path.basename(local_path) in existing_items:
            print(f'Skipping {local_path} as it already exists in Dropbox')
        else:
            with open(local_path, "rb") as f:
                dbx.files_upload(f.read(), dropbox_path)
                print(f'Uploaded {local_path} to {dropbox_path}')

# Define the local directory to upload
local_dir = 'crypto'  # Change to your source directory
dropbox_target = '/CMC_Token_Price DATA/crypto'  # Change to your target Dropbox folder

# Start the recursive upload process
upload_to_dropbox(local_dir, dropbox_target)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Skipping crypto/ohlcv_active_assets/25330_ANIMA_2023-06-07T19:40:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25331_PEPEAI_2023-05-16T19:25:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25333_MURATIAI_2023-05-17T01:40:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25337_GRAI_2023-05-17T18:40:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25347_OIL_2023-05-17T12:40:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25354_AA_2023-05-24T15:40:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_assets/25359_PEPE_2023-05-17T19:25:00.000Z_2024-10-09T04:05:00.000Z.csv as it already exists in Dropbox
Skipping crypto/ohlcv_active_

***
# Filtering data

In [ ]:
import zipfile

def create_zip_from_ids(directory, ids, zip_filename):
    """Creates a zip file with CSV files based on a list of IDs.

    Args:
    - directory: The path to the directory containing the CSV files.
    - ids: A list of IDs to search for in the filenames.
    - zip_filename: The output zip filename.
    """
    with zipfile.ZipFile(zip_filename, 'a') as zipf:
        for file_name in os.listdir(directory):
            # Check if the file starts with any of the IDs followed by "_"
            for id_ in ids:
                if file_name.startswith(f"{id_}_") and file_name.endswith('.csv'):
                    # Get full path of the file
                    file_path = os.path.join(directory, file_name)
                    # Add file to the zip archive
                    zipf.write(file_path, arcname=file_name)
                    print(f"Added {file_name} to the zip archive.")

# Example usage
directory = 'crypto/ohclv_inactive_assets'  # Directory containing the CSV files
ids = pd.read_excel('data/filtered_crypto_tracked_assets.xlsx')['id']
zip_filename = 'filtered_crypto_ohlcv.zip'  # Name of the output zip file

# Create the zip file
create_zip_from_ids(directory, ids, zip_filename)

Added 1229_DGD_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 6650_NFT_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 11734_LTY_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 15557_MOM_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 15589_MILK_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 15761_GEAR_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 15907_BITCOIN_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 15985_MONGOOSE_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 16832_WEB3_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 17566_ROAR_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive.
Added 19985_SLAM_2019-10-09T04:00:00.000Z_2024-10-09T04:00:00.000Z.csv to the zip archive

In [ ]:
def aggregate_csv_files(directory, output_file):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Initialize an empty list to store DataFrames
    df_list = []

    # Loop through each CSV file and read it
    for file in csv_files:
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file)
        # Append the DataFrame to the list
        df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    aggregated_df = pd.concat(df_list, ignore_index=True)

    # Save the aggregated DataFrame to a new CSV file
    aggregated_df.to_csv(output_file, index=False)
    print(f"All CSV files have been aggregated into {output_file}")

# Example usage:
directory = output_dir
output_file = 'data/request_hourly_tao.csv'

aggregate_csv_files(directory, output_file)

All CSV files have been aggregated into data/request_hourly_tao.csv


In [ ]:
request = pd.read_csv(output_file)
request.head()

,asset_id,name,symbol,time_open,time_close,time_high,time_low,open,high,low,close,volume,market_cap
0,9154,Elastic Governance,EGT,2021-04-07T18:00:00.000Z,2021-04-07T18:59:59.999Z,2021-04-07T18:20:17.000Z,2021-04-07T18:56:15.000Z,283.140603,283.195207,276.239713,276.569804,2130186.08,0.0
1,9154,Elastic Governance,EGT,2021-04-07T19:00:00.000Z,2021-04-07T19:59:59.999Z,2021-04-07T19:58:11.000Z,2021-04-07T19:23:10.000Z,276.235789,277.219169,273.092906,276.830412,2026836.30,0.0
2,9154,Elastic Governance,EGT,2021-04-07T20:00:00.000Z,2021-04-07T20:59:59.999Z,2021-04-07T20:59:11.000Z,2021-04-07T20:29:10.000Z,277.001679,293.531769,276.025636,293.531769,1882365.27,0.0
3,9154,Elastic Governance,EGT,2021-04-07T21:00:00.000Z,2021-04-07T21:59:59.999Z,2021-04-07T21:55:17.000Z,2021-04-07T21:11:13.000Z,293.394113,295.382633,293.204691,295.048188,1820559.55,0.0
4,9154,Elastic Governance,EGT,2021-04-07T22:00:00.000Z,2021-04-07T22:59:59.999Z,2021-04-07T22:47:10.000Z,2021-04-07T22:05:17.000Z,295.090310,297.034671,294.371737,296.660393,NaN,0.0


In [ ]:
a = request[['asset_id', 'symbol']].drop_duplicates()

In [ ]:
b = filtered_assets[['UCID','CMC_ticker']].drop_duplicates().dropna()

In [ ]:
c = pd.merge(a, b, left_on='asset_id', right_on='UCID')

In [ ]:
c[c['symbol']!=c['CMC_ticker']]

,asset_id,symbol,UCID,CMC_ticker
72,11156,ETHDYDX,11156,DYDX
202,7977,DUCK,7977,COL


In [ ]:
with open('/content/Specification.md', 'w') as f:
    f.write('# Specification\n\nThis is the content of your specification.')

### Data Specification Document

---

#### 1. **File Naming Convention:**

Each file in this dataset follows the naming convention:

```
{id}_{symbol}_{first_historical_data}_{last_historical_data}.csv
```

Where:
- **`{id}`**: Unique identifier for the data collection process (e.g., `33328`).
- **`{symbol}`**: The ticker symbol representing the cryptocurrency asset (e.g., `MOONBIX`).
- **`{first_historical_data}`**: The timestamp of the first recorded data point in the file, in ISO 8601 format (e.g., `2024-10-07T08:40:00.000Z`).
- **`{last_historical_data}`**: The timestamp of the last recorded data point in the file, in ISO 8601 format (e.g., `2024-10-09T10:40:00.000Z`).

**Example Filename**:
```
33328_MOONBIX_2024-10-07T08:40:00.000Z_2024-10-09T10:40:00.000Z.csv
```

---

#### 2. **Data Collection Details:**

- **Collection Period**: Data is collected between the timestamps `{first_historical_data}` and `{last_historical_data}` as indicated in the file name.
- **Collection Frequency**: Data points are recorded dynamically throughout the collection period based on market activity.

---

#### 3. **Data Structure:**

Each CSV file contains the following columns, which represent specific data points for the corresponding cryptocurrency asset:

| **Column Name** | **Data Type** | **Description**                                                                                     |
|-----------------|---------------|-----------------------------------------------------------------------------------------------------|
| `time_open`     | `datetime`    | Timestamp of when the market opened for this specific period (ISO 8601 format).                     |
| `time_close`    | `datetime`    | Timestamp of when the market closed for this specific period (ISO 8601 format).                     |
| `time_high`     | `datetime`    | Timestamp when the highest price was recorded during the period (ISO 8601 format).                  |
| `time_low`      | `datetime`    | Timestamp when the lowest price was recorded during the period (ISO 8601 format).                   |
| `open`          | `float`       | Price of the asset at the beginning of the period.                                                   |
| `high`          | `float`       | Highest price of the asset during the period.                                                        |
| `low`           | `float`       | Lowest price of the asset during the period.                                                         |
| `close`         | `float`       | Price of the asset at the end of the period.                                                         |
| `volume`        | `float`       | Total volume traded during the period.                                                              |
| `market_cap`    | `float`       | Market capitalization of the asset at the end of the period (if available).                         |

---

#### 4. **Data Format:**

- **Time Format**: All timestamps are in ISO 8601 format (`YYYY-MM-DDTHH:MM:SS.sssZ`).
- **Numerical Values**: Prices (`open`, `high`, `low`, `close`), `volume`, and `market_cap` are represented as floating-point numbers.

---

#### 5. **Example Data Entry:**

```
time_open,time_close,time_high,time_low,open,high,low,close,volume,market_cap
2024-10-08T00:00:00.000Z,2024-10-08T23:59:59.999Z,2024-10-08T09:59:00.000Z,2024-10-08T14:30:00.000Z,1.26E-10,1.36E-10,5.03E-11,8.43E-11,3678.31,0
```

---

#### 6. **File Content Details:**

- **Time Open (`time_open`)**: Start time of the period.
- **Time Close (`time_close`)**: End time of the period.
- **Time High (`time_high`)**: Specific time when the highest price occurred during the period.
- **Time Low (`time_low`)**: Specific time when the lowest price occurred during the period.
- **Open**: Asset price at the opening of the period.
- **High**: Maximum price reached during the period.
- **Low**: Minimum price reached during the period.
- **Close**: Asset price at the end of the period.
- **Volume**: Total volume traded during the period.
- **Market Cap**: Total market capitalization of the asset at the close of the period.

---

#### 7. **Notes:**
- This data represents historical OHLCV (Open, High, Low, Close, Volume) values for the specified cryptocurrency.
- Market capitalization data may not always be available (`market_cap` may be zero).

This document provides a clear understanding of the data structure, collection period, and format of the delivered dataset.